In [1]:
import numpy as np
import pandas as pd

# Load item and rating info

In [184]:
items = pd.read_excel('office.xlsx', sheet_name='Items', index_col=0)

In [181]:
ratings = pd.read_excel('office.xlsx', sheet_name='Ratings', index_col=0).stack().reset_index()
ratings.columns = ['Product', 'User', 'Rating']
ratings['User'] = ratings['User'].astype(int)
ratings['Rating'] = ratings['Rating'].astype(int)
ratings.sort_values(by=['User', 'Product'], inplace=True)
ratings.head()

,Product,User,Rating
37,72,64,4
426,1240,64,5
564,1300,64,4
573,1317,64,5
620,1327,64,3


In [182]:
good_ratings = ratings.loc[(ratings['Rating']==4) | (ratings['Rating']==5)]

# Load predicted rating

In [244]:
cbf = pd.read_excel('office.xlsx', sheet_name='CBF', index_col=0)
itemitem = pd.read_excel('office.xlsx', sheet_name='Item-Item', index_col=0)
useruser = pd.read_excel('office.xlsx', sheet_name='User-User', index_col=0)

In [247]:
def get_rec(predicted_ratings):
    predicted_ratings.columns = predicted_ratings.columns.astype(int)
    rec = predicted_ratings.apply(lambda user: user.nlargest(5).index)
    rec = rec.stack().reset_index(level=1)
    rec.columns = ['User', 'Product']
    rec.sort_values(by=['User', 'Product'], inplace=True)
    return rec

In [249]:
def get_metrics(rec, good_ratings, items):
    precision = len(pd.merge(rec, good_ratings[['Product', 'User']])) / 500
    price_div = items.loc[rec['Product'], 'Price'].std()
    avail_avg = items.loc[rec['Product'], 'Availability'].mean()
    return precision, price_div, avail_avg

# Get performance results for each algorithm

In [250]:
get_metrics(get_rec(cbf), good_ratings, items)

(0.104, 31.761838890123908, 0.5853634540762697)

In [251]:
get_metrics(get_rec(itemitem), good_ratings, items)

(0.214, 36.2763727260406, 0.6133728647426743)

In [252]:
get_metrics(get_rec(useruser.dropna(axis=1)), good_ratings, items)

(0.52, 40.496623332098146, 0.6964598020812371)